In [85]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import os

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import warnings
warnings.filterwarnings('ignore')

In [86]:
# Important landmarks for plank
landmarks = [
    'LEFT_SHOULDER',
    'RIGHT_SHOULDER',
    'LEFT_ELBOW',
    'RIGHT_ELBOW',
    'LEFT_WRIST',
    'RIGHT_WRIST',
    'LEFT_HIP',
    'RIGHT_HIP',
    'LEFT_KNEE',
    'RIGHT_KNEE',
    'LEFT_ANKLE',
    'RIGHT_ANKLE',
    'LEFT_HEEL',
    'RIGHT_HEEL',
    'LEFT_FOOT_INDEX',
    'RIGHT_FOOT_INDEX',
]

headers = ['label'] 

# Generate all columns of the data frame
for landmark in landmarks:
    headers += [f'{landmark.lower()}_x', f'{landmark.lower()}_y', f'{landmark.lower()}_z', f'{landmark.lower()}_v']
print(f'Number of columns: {len(headers)}')

empty_df = pd.DataFrame(columns=headers)

Number of columns: 65


In [87]:
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

I0000 00:00:1718159490.085101 10552156 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
W0000 00:00:1718159490.259150 10579215 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718159490.330070 10579215 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [88]:
def create_image_data(files, features, label, df):
    df = df.copy()
    
    for file in files:
        image = mp.Image.create_from_file(file)

        # Extract the landmarks 
        pose_landmarks = detector.detect(image).pose_landmarks[0]

        # Only keep the landmarks of interest
        landmarks = []
        for feature in features:
            landmark = pose_landmarks[mp.solutions.pose.PoseLandmark[feature].value]
            landmarks.append([landmark.x, landmark.y, landmark.z, landmark.visibility])
        landmarks = list(np.array(landmarks).flatten())

        # Add the label to the landmarks
        sample = [label] + landmarks
        sample = pd.Series(sample, index=df.columns)

        # Add the sample to the data frame
        df = pd.concat([df, sample.to_frame().T], ignore_index=True)
    
    return df

In [89]:
def create_video_data(files, frame_capture_rate, features, label, df):
    df = df.copy()

    for file in files:
        frame_number = 0

        with mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            video = cv2.VideoCapture(file)
        
            while video.isOpened():
                ret, frame = video.read() 
                
                if not ret:
                    break
                
                if frame_number % frame_capture_rate == 0: # Only process every nth frame
                    pose_landmarks = pose.process(frame).pose_landmarks.landmark

                    # Only keep the landmarks of interest
                    landmarks = []
                    for feature in features:
                        landmark = pose_landmarks[mp.solutions.pose.PoseLandmark[feature].value]
                        landmarks.append([landmark.x, landmark.y, landmark.z, landmark.visibility])

                    # Add the label to the landmarks
                    landmarks = list(np.array(landmarks).flatten())
                    sample = [label] + landmarks

                    # Add the sample to the data frame
                    if len(sample) == len(df.columns):
                        sample = pd.Series(sample, index=df.columns)
                        df = pd.concat([df, sample.to_frame().T], ignore_index=True)
            
                frame_number += 1
            video.release()
       
    return df

# **Generate Test Data from Images**

In [90]:
image_data_path = 'data/plank/train/image'
video_data_path = 'data/plank/train/video'

In [91]:
correct_files = [os.path.join(image_data_path, 'correct', file) for file in os.listdir(os.path.join(image_data_path, 'correct'))]
print('Number of correct samples:', len(correct_files))

correct_df = create_image_data(correct_files, landmarks, 'c', empty_df)
print('Correct data frame shape:', correct_df.shape)
correct_df.head()

Number of correct samples: 16
Correct data frame shape: (16, 65)


,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left_elbow_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,c,0.755873,0.37134,0.142172,0.999981,0.780393,0.374476,-0.232091,0.999961,0.740323,...,-0.034949,0.987611,0.105399,0.816902,0.142222,0.768547,0.102585,0.836181,-0.154142,0.988942
1,c,0.204851,0.488668,-0.18609,0.999937,0.229784,0.380936,0.243581,0.999875,0.221035,...,0.165992,0.500833,0.893337,0.800755,-0.156181,0.965273,0.883106,0.752967,0.10702,0.515329
2,c,0.799452,0.323116,0.251574,0.999886,0.813881,0.419781,-0.20176,0.999942,0.783975,...,-0.084245,0.989843,0.120464,0.835478,0.194059,0.81017,0.086306,0.890535,-0.209508,0.990062
3,c,0.24442,0.371601,-0.261188,0.999804,0.264717,0.310763,0.295749,0.999603,0.251703,...,0.270052,0.312724,0.891732,0.877317,-0.229655,0.939585,0.886772,0.819571,0.159524,0.374121
4,c,0.243312,0.473431,-0.390952,0.999932,0.257345,0.371256,0.281002,0.999812,0.270554,...,0.427192,0.434707,0.928693,0.914889,-0.077621,0.873237,0.910039,0.877281,0.310827,0.421052


In [92]:
low_image_files = [os.path.join(image_data_path, 'low', file) for file in os.listdir(os.path.join(image_data_path, 'low'))]
print('Number of low samples:', len(low_image_files))
print(low_image_files)

low_image_df = create_image_data(low_image_files, landmarks, 'l', empty_df)
print('Low data frame shape:', low_image_df.shape)
low_image_df.head()

Number of low samples: 16
['data/plank/train/image/low/Screen Shot 2024-06-10 at 1.55.00 PM.png', 'data/plank/train/image/low/Screen Shot 2024-06-10 at 1.50.05 PM.png', 'data/plank/train/image/low/Screen Shot 2024-06-10 at 2.00.00 PM.png', 'data/plank/train/image/low/Screen Shot 2024-06-10 at 1.57.48 PM.png', 'data/plank/train/image/low/Screen Shot 2024-06-10 at 1.55.49 PM.png', 'data/plank/train/image/low/Screen Shot 2024-06-10 at 1.54.36 PM.png', 'data/plank/train/image/low/rawImage.jpg', 'data/plank/train/image/low/Screen Shot 2024-06-10 at 1.41.08 PM.png', 'data/plank/train/image/low/00000324.jpg', 'data/plank/train/image/low/Screenshot 2024-06-11 at 1.45.48\u202fAM.png', 'data/plank/train/image/low/00000132.jpg', 'data/plank/train/image/low/Screen Shot 2024-06-10 at 1.55.35 PM.png', 'data/plank/train/image/low/Screen Shot 2024-06-10 at 1.48.12 PM.png', 'data/plank/train/image/low/Screen Shot 2024-06-10 at 1.57.30 PM.png', 'data/plank/train/image/low/Screen Shot 2024-06-10 at 1.57.

,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left_elbow_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,l,0.207304,0.377525,-0.207559,0.999781,0.22457,0.314518,0.175727,0.999465,0.247943,...,0.159017,0.306151,0.889678,0.822372,-0.158673,0.917595,0.855868,0.764802,0.103131,0.276856
1,l,0.772448,0.237865,0.200576,0.999983,0.78795,0.355173,-0.186457,0.999965,0.750179,...,-0.057278,0.997923,0.126143,0.771611,0.254396,0.965288,0.097072,0.862351,-0.1261,0.998315
2,l,0.775156,0.448662,0.214646,0.999879,0.807127,0.514001,-0.125711,0.99997,0.768268,...,-0.030357,0.950442,0.128417,0.896496,0.23014,0.555288,0.109461,0.925922,-0.06684,0.938204
3,l,0.814106,0.119444,0.207929,0.999981,0.813866,0.160392,-0.291196,0.999981,0.770568,...,-0.084347,0.992579,0.092681,0.66186,0.2286,0.903576,0.056413,0.804102,-0.232467,0.993057
4,l,0.141306,0.273406,-0.271097,0.999978,0.204585,0.239962,0.276126,0.999864,0.211544,...,0.226206,0.474378,0.956583,0.86284,-0.222333,0.969115,0.951063,0.73628,0.147898,0.444942


In [93]:
high_image_files = [os.path.join(image_data_path, 'high', file) for file in os.listdir(os.path.join(image_data_path, 'high'))]
print('Number of high samples:', len(high_image_files))

high_image_df = create_image_data(high_image_files, landmarks, 'h', empty_df)
print('High data frame shape:', high_image_df.shape)
high_image_df.head()

Number of high samples: 15
High data frame shape: (15, 65)


,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left_elbow_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,h,0.761738,0.473951,0.191882,0.999982,0.799332,0.524085,-0.150481,0.999991,0.771962,...,-0.017896,0.994403,0.1704,0.7477,0.210187,0.778022,0.166921,0.784381,-0.087047,0.991719
1,h,0.278051,0.442553,-0.23253,0.999546,0.273764,0.321988,0.087097,0.999399,0.27229,...,0.40498,0.035663,0.873996,0.69597,0.271223,0.288011,0.857829,0.689818,0.391697,0.029006
2,h,0.411856,0.369617,-0.534391,0.999907,0.367767,0.370753,-0.063759,0.999867,0.431441,...,0.769382,0.545164,0.808952,0.713842,0.326493,0.971651,0.746445,0.683177,0.697973,0.753872
3,h,0.789327,0.44269,0.237454,0.999969,0.808275,0.547472,-0.208431,0.99999,0.779033,...,-0.138777,0.999456,0.196323,0.805368,0.214246,0.947438,0.12813,0.891945,-0.246104,0.999249
4,h,0.789116,0.428211,0.242215,0.999969,0.811261,0.544843,-0.184502,0.999988,0.764635,...,-0.097662,0.998134,0.193621,0.811241,0.236521,0.887194,0.140698,0.890875,-0.199125,0.997404


# **Generate Train Data from Videos**

In [109]:
correct_video_files = [os.path.join(video_data_path, 'correct', file) for file in os.listdir(os.path.join(video_data_path, 'correct'))]
print('Number of correct samples:', len(correct_video_files))

correct_video_df = create_video_data(correct_video_files, 10, landmarks, 'c', empty_df)
print('Correct video data frame shape:', correct_video_df.shape)
correct_video_df.head()


Number of correct samples: 1


I0000 00:00:1718160051.285943 10552156 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
W0000 00:00:1718160051.399021 10586671 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718160051.406173 10586673 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Correct video data frame shape: (48, 65)


,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left_elbow_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,c,0.801685,0.430241,0.296902,0.998821,0.827469,0.582133,-0.183411,0.999906,0.748023,...,-0.154902,0.983322,0.143478,0.845839,0.05891,0.769544,0.110715,0.893111,-0.265165,0.983547
1,c,0.802082,0.430561,0.28546,0.99872,0.827264,0.584164,-0.195572,0.999894,0.747761,...,-0.164177,0.982067,0.142749,0.846277,0.114806,0.761573,0.110686,0.893059,-0.272214,0.9824
2,c,0.802444,0.430771,0.279871,0.998601,0.82727,0.584609,-0.203166,0.999884,0.747667,...,-0.170876,0.980981,0.14373,0.846004,0.223533,0.752082,0.110943,0.893118,-0.277951,0.981409
3,c,0.802667,0.431337,0.276838,0.998497,0.827418,0.584604,-0.208978,0.999876,0.754724,...,-0.173829,0.980088,0.149915,0.845879,0.277794,0.741793,0.113738,0.893274,-0.281129,0.980459
4,c,0.803012,0.431589,0.275737,0.998417,0.827729,0.58448,-0.208883,0.99987,0.75864,...,-0.187295,0.979514,0.154529,0.845735,0.299705,0.733577,0.115472,0.893657,-0.294216,0.97983


In [110]:
low_video_files = [os.path.join(video_data_path, 'low', file) for file in os.listdir(os.path.join(video_data_path, 'low'))]
print('Number of low samples:', len(low_video_files))

low_video_df = create_video_data(low_video_files, 10, landmarks, 'l', empty_df)
print('Low video data frame shape:', low_video_df.shape)
low_video_df.head()

Number of low samples: 1


I0000 00:00:1718160052.743346 10552156 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
W0000 00:00:1718160052.837158 10586704 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718160052.843514 10586704 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Low video data frame shape: (30, 65)


,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left_elbow_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,l,0.787686,0.494229,0.217198,0.996827,0.810987,0.595809,-0.245476,0.99959,0.729904,...,-0.133849,0.95664,0.093979,0.89564,0.155599,0.83682,0.056419,0.92323,-0.253256,0.970347
1,l,0.789331,0.496999,0.223793,0.996749,0.809999,0.594668,-0.240316,0.999579,0.729011,...,-0.123211,0.956528,0.108773,0.887954,0.155602,0.833398,0.062953,0.927454,-0.230439,0.969582
2,l,0.790397,0.49887,0.227606,0.996645,0.809513,0.593907,-0.239349,0.999565,0.728188,...,-0.12203,0.956358,0.113718,0.883665,0.164845,0.8301,0.065416,0.929548,-0.228084,0.968768
3,l,0.79122,0.500219,0.229247,0.996554,0.809252,0.59352,-0.237964,0.999555,0.728243,...,-0.122644,0.956118,0.117074,0.881269,0.173725,0.826156,0.066951,0.931901,-0.228784,0.967886
4,l,0.791145,0.500632,0.2287,0.996493,0.807996,0.593483,-0.23407,0.999554,0.726589,...,-0.127593,0.957041,0.120188,0.880765,0.192795,0.826329,0.067667,0.9348,-0.231056,0.968254


In [111]:
high_video_files = [os.path.join(video_data_path, 'high', file) for file in os.listdir(os.path.join(video_data_path, 'high'))]
print('Number of high samples:', len(high_video_files))

high_video_df = create_video_data(high_video_files, 10, landmarks, 'h', empty_df)
print('High video data frame shape:', high_video_df.shape)
high_video_df.head()

Number of high samples: 1


I0000 00:00:1718160053.789105 10552156 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
W0000 00:00:1718160053.885924 10586755 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718160053.891874 10586761 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


High video data frame shape: (39, 65)


,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left_elbow_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,h,0.800204,0.465185,0.30256,0.996815,0.818665,0.575554,-0.224919,0.999832,0.775807,...,-0.18362,0.986964,0.154245,0.832613,0.298837,0.717325,0.107849,0.909471,-0.300584,0.987417
1,h,0.800201,0.461336,0.318944,0.99704,0.818647,0.575496,-0.226528,0.999845,0.76819,...,-0.315155,0.987002,0.154239,0.83539,0.421889,0.721194,0.098807,0.913059,-0.426246,0.987578
2,h,0.800204,0.457981,0.323685,0.997231,0.818464,0.575321,-0.227448,0.999856,0.766121,...,-0.316248,0.986951,0.154253,0.836239,0.41652,0.723713,0.098612,0.915067,-0.431316,0.987656
3,h,0.800517,0.454637,0.322045,0.997412,0.817682,0.575318,-0.227576,0.999866,0.766439,...,-0.31275,0.987046,0.154351,0.838987,0.433947,0.728291,0.099154,0.91908,-0.42943,0.987894
4,h,0.80072,0.452388,0.320428,0.997574,0.817417,0.574385,-0.229045,0.999876,0.766838,...,-0.315066,0.987273,0.153857,0.841624,0.427457,0.735746,0.098809,0.918944,-0.433057,0.988256


# **Sample Train Data**

In [112]:
plank_data = pd.read_csv('data/plank/train/NgoQuocBao1010_plank_data.csv')

plank_data['label'] = plank_data['label'].str.lower()

# Separate the classes
low_data = plank_data[plank_data['label'] == 'l']
high_data = plank_data[plank_data['label'] == 'h']
correct_data = plank_data[plank_data['label'] == 'c']

# Randomly sample 50 from each class
sample_size = 50
sample_low = low_data.sample(n=sample_size, random_state=42)
sample_high = high_data.sample(n=sample_size, random_state=42)
sample_correct = correct_data.sample(n=sample_size, random_state=42)

# Remove the nose landmarks features
sampled_data = pd.concat([sample_low, sample_high, sample_correct])
sampled_data = sampled_data.drop(columns=['nose_x', 'nose_y', 'nose_z', 'nose_v'])
print('Unique labels:', sampled_data['label'].value_counts())

print('Sampled data shape:', sampled_data.shape)
sampled_data.head()

Unique labels: label
l    50
h    50
c    50
Name: count, dtype: int64
Sampled data shape: (150, 65)


,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left_elbow_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
18240,l,0.754218,0.370173,0.241762,0.997989,0.780318,0.417177,-0.166239,0.999572,0.735011,...,-0.127144,0.932999,0.096163,0.691377,0.083459,0.685030,0.057381,0.715348,-0.221229,0.922233
24266,l,0.243449,0.715532,-0.143181,0.999678,0.253665,0.694626,0.174842,0.998643,0.259328,...,0.154528,0.811783,0.607543,0.870063,-0.113310,0.976028,0.594334,0.855911,0.114326,0.900786
8651,l,0.287879,0.454376,-0.245247,0.999568,0.315570,0.415902,0.120281,0.998195,0.289889,...,0.264711,0.713180,0.903022,0.718910,0.002256,0.938012,0.870515,0.697409,0.245445,0.751438
23513,l,0.255347,0.706452,-0.151369,0.999481,0.263588,0.694241,0.147905,0.998031,0.262750,...,0.129128,0.711834,0.621445,0.869514,-0.090647,0.945976,0.611136,0.857050,0.083203,0.783405
15179,l,0.177219,0.293821,-0.198248,0.999584,0.208486,0.272323,0.269599,0.997566,0.194945,...,0.188373,0.785181,0.957764,0.624996,-0.182459,0.980558,0.918317,0.609237,0.105205,0.891345


# **Combine all data**

In [114]:
final_data = pd.concat([correct_df, low_image_df, high_image_df, correct_video_df, low_video_df, high_video_df, sampled_data], ignore_index=True)
# final_data = pd.concat([correct_df, low_image_df, high_image_df, sampled_data], ignore_index=True)
print('Final data frame shape:', final_data.shape)
print('Unique labels:', final_data['label'].value_counts())
final_data.head()


Final data frame shape: (314, 65)
Unique labels: label
c    114
h    104
l     96
Name: count, dtype: int64


,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left_elbow_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,c,0.755873,0.37134,0.142172,0.999981,0.780393,0.374476,-0.232091,0.999961,0.740323,...,-0.034949,0.987611,0.105399,0.816902,0.142222,0.768547,0.102585,0.836181,-0.154142,0.988942
1,c,0.204851,0.488668,-0.18609,0.999937,0.229784,0.380936,0.243581,0.999875,0.221035,...,0.165992,0.500833,0.893337,0.800755,-0.156181,0.965273,0.883106,0.752967,0.10702,0.515329
2,c,0.799452,0.323116,0.251574,0.999886,0.813881,0.419781,-0.20176,0.999942,0.783975,...,-0.084245,0.989843,0.120464,0.835478,0.194059,0.81017,0.086306,0.890535,-0.209508,0.990062
3,c,0.24442,0.371601,-0.261188,0.999804,0.264717,0.310763,0.295749,0.999603,0.251703,...,0.270052,0.312724,0.891732,0.877317,-0.229655,0.939585,0.886772,0.819571,0.159524,0.374121
4,c,0.243312,0.473431,-0.390952,0.999932,0.257345,0.371256,0.281002,0.999812,0.270554,...,0.427192,0.434707,0.928693,0.914889,-0.077621,0.873237,0.910039,0.877281,0.310827,0.421052


In [115]:
final_data.to_csv('data/plank/train/train_data.csv', index=False)

# **Generate Test Data**

In [117]:
image_data_path = 'data/plank/test/image'

In [118]:
correct_files = [os.path.join(image_data_path, 'correct', file) for file in os.listdir(os.path.join(image_data_path, 'correct'))]
print('Number of correct samples:', len(correct_files))

correct_df = create_image_data(correct_files, landmarks, 'c', empty_df)
print('Correct data frame shape:', correct_df.shape)
correct_df.head()

Number of correct samples: 5
Correct data frame shape: (5, 65)


,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left_elbow_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,c,0.714614,0.519723,0.201462,0.999987,0.729075,0.541016,-0.250548,0.999981,0.704564,...,-0.08212,0.994909,0.164946,0.719515,0.129273,0.934534,0.153084,0.73145,-0.185452,0.995518
1,c,0.787049,0.331529,0.27855,0.99994,0.821669,0.378449,-0.295977,0.999963,0.778385,...,0.089337,0.995814,0.102723,0.916706,0.330069,0.8797,0.085149,0.954046,-0.041968,0.994629
2,c,0.269111,0.595271,-0.430963,0.999986,0.244112,0.531469,0.152096,0.999922,0.388862,...,0.691453,0.507217,0.904117,0.811899,0.00375,0.976133,0.862345,0.772885,0.614237,0.556602
3,c,0.725329,0.411023,0.241991,0.999954,0.745769,0.44134,-0.214798,0.999894,0.730552,...,-0.032006,0.953844,0.111073,0.849412,0.150137,0.593378,0.099482,0.871862,-0.160276,0.942345
4,c,0.367654,0.669358,-0.218309,0.999888,0.374715,0.666738,0.179499,0.999387,0.363128,...,0.302068,0.319193,0.753048,0.852669,-0.085756,0.908649,0.733268,0.842049,0.225437,0.343772


In [119]:
low_image_files = [os.path.join(image_data_path, 'low', file) for file in os.listdir(os.path.join(image_data_path, 'low'))]
print('Number of low samples:', len(low_image_files))
print(low_image_files)

low_image_df = create_image_data(low_image_files, landmarks, 'l', empty_df)
print('Low data frame shape:', low_image_df.shape)
low_image_df.head()

Number of low samples: 5
['data/plank/test/image/low/Screenshot 2024-06-11 at 1.51.16\u202fAM.png', 'data/plank/test/image/low/Screenshot 2024-06-11 at 1.43.22\u202fAM.png', 'data/plank/test/image/low/Screenshot 2024-06-11 at 1.44.31\u202fAM.png', 'data/plank/test/image/low/Screenshot 2024-06-11 at 1.45.13\u202fAM.png', 'data/plank/test/image/low/00000341.jpg']
Low data frame shape: (5, 65)


,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left_elbow_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,l,0.77896,0.393427,0.176537,0.999959,0.80428,0.479416,-0.257508,0.999933,0.765966,...,0.031335,0.964869,0.058047,0.824998,0.205295,0.527107,0.051062,0.835042,-0.030396,0.952871
1,l,0.201275,0.413116,-0.169261,0.999772,0.230501,0.349821,0.135527,0.999751,0.206352,...,0.129688,0.155028,0.879079,0.842187,-0.060816,0.788053,0.84946,0.793299,0.078431,0.17656
2,l,0.168011,0.48412,-0.207193,0.999966,0.205339,0.419365,0.345297,0.999914,0.196024,...,0.285441,0.43003,0.912898,0.821241,-0.240395,0.899614,0.885103,0.760242,0.203536,0.328393
3,l,0.211965,0.41462,-0.21837,0.999606,0.227408,0.3459,0.183866,0.999125,0.24713,...,0.204968,0.317796,0.884108,0.89239,-0.13313,0.915458,0.861084,0.841811,0.151035,0.288934
4,l,0.237173,0.427308,-0.26866,0.999757,0.271587,0.393058,0.158602,0.999679,0.255956,...,0.332239,0.16359,0.889659,0.657582,-0.107831,0.823099,0.856326,0.615896,0.288328,0.172183


In [120]:
high_image_files = [os.path.join(image_data_path, 'high', file) for file in os.listdir(os.path.join(image_data_path, 'high'))]
print('Number of high samples:', len(high_image_files))

high_image_df = create_image_data(high_image_files, landmarks, 'h', empty_df)
print('High data frame shape:', high_image_df.shape)
high_image_df.head()

Number of high samples: 5
High data frame shape: (5, 65)


,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left_elbow_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,h,0.751467,0.455332,0.220172,0.999947,0.780532,0.53767,-0.21004,0.99997,0.741444,...,-0.029889,0.993391,0.098355,0.829018,0.184237,0.753898,0.095989,0.842453,-0.108654,0.989894
1,h,0.200844,0.404164,-0.207464,0.99966,0.208228,0.319205,0.133639,0.999414,0.227917,...,0.253891,0.149029,0.863694,0.775571,-0.12562,0.942414,0.816165,0.728578,0.209173,0.166391
2,h,0.231539,0.556217,-0.170029,0.999176,0.254021,0.553137,0.246847,0.998288,0.203396,...,0.072102,0.005562,1.07722,0.833716,-0.05716,0.056935,1.056962,0.786195,0.024342,0.002395
3,h,0.2471,0.515461,-0.187773,0.999542,0.262143,0.46212,0.134628,0.999293,0.229271,...,0.252077,0.135897,0.90262,0.718165,0.047468,0.850898,0.883099,0.689643,0.204763,0.134957
4,h,0.204657,0.505942,-0.170077,0.999034,0.216457,0.418698,0.141148,0.996845,0.163369,...,0.202821,0.023992,0.867318,0.851169,0.00523,0.275846,0.847274,0.822991,0.154716,0.013837


In [121]:
final_data = pd.concat([correct_df, low_image_df, high_image_df], ignore_index=True)
print('Final data frame shape:', final_data.shape)
print('Unique labels:', final_data['label'].value_counts())
final_data.head()


Final data frame shape: (15, 65)
Unique labels: label
c    5
l    5
h    5
Name: count, dtype: int64


,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left_elbow_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,c,0.714614,0.519723,0.201462,0.999987,0.729075,0.541016,-0.250548,0.999981,0.704564,...,-0.08212,0.994909,0.164946,0.719515,0.129273,0.934534,0.153084,0.73145,-0.185452,0.995518
1,c,0.787049,0.331529,0.27855,0.99994,0.821669,0.378449,-0.295977,0.999963,0.778385,...,0.089337,0.995814,0.102723,0.916706,0.330069,0.8797,0.085149,0.954046,-0.041968,0.994629
2,c,0.269111,0.595271,-0.430963,0.999986,0.244112,0.531469,0.152096,0.999922,0.388862,...,0.691453,0.507217,0.904117,0.811899,0.00375,0.976133,0.862345,0.772885,0.614237,0.556602
3,c,0.725329,0.411023,0.241991,0.999954,0.745769,0.44134,-0.214798,0.999894,0.730552,...,-0.032006,0.953844,0.111073,0.849412,0.150137,0.593378,0.099482,0.871862,-0.160276,0.942345
4,c,0.367654,0.669358,-0.218309,0.999888,0.374715,0.666738,0.179499,0.999387,0.363128,...,0.302068,0.319193,0.753048,0.852669,-0.085756,0.908649,0.733268,0.842049,0.225437,0.343772


In [122]:
final_data.to_csv('data/plank/test/test_data.csv', index=False)